In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice'

In [31]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str 
    target_column:str
    subsample: float
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int 
    max_depth: int
    learning_rate: float    

In [32]:
from src.AutoMobilePriceRegression.constants import *
from src.AutoMobilePriceRegression.utils.common import read_yaml, create_directories

In [33]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) -> None:
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])  


    
    def get_training_config(self)-> TrainingConfig:
        config=self.config.training
        params=self.params.GradientBoostingRegressor
        schema=self.schema.TARGET_COLUMN
        

        create_directories([config.root_dir])

        training_config = TrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name,
            subsample=params.subsample,
            n_estimators=params.n_estimators,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            max_depth=params.max_depth,
            learning_rate=params.learning_rate    

        )

        return training_config

In [34]:
import os
import urllib.request as request
from sklearn.ensemble import GradientBoostingRegressor
import time
from sklearn.metrics import r2_score
from sklearn.metrics import r2_score,mean_absolute_percentage_error
import pickle,joblib
import pandas as pd

In [35]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config= config

    def initiate_Training(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        x_train,y_train,x_test,y_test=(
            train_data.drop(self.config.target_column,axis=1),train_data[self.config.target_column],
            test_data.drop(self.config.target_column,axis=1),test_data[self.config.target_column]
        )    

        model=GradientBoostingRegressor(
                subsample=self.config.subsample,
                n_estimators=self.config.n_estimators,
                min_samples_split=self.config.min_samples_split,
                min_samples_leaf=self.config.min_samples_leaf,
                max_depth=self.config.max_depth,
                learning_rate=self.config.learning_rate
        )
        model.fit(x_train,y_train)

        joblib.dump(model,os.path.join(self.config.root_dir,self.config.model_name))

In [36]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.initiate_Training()
except Exception as e:
    raise e

[2024-09-07 00:45:03,908: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-07 00:45:03,911: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-07 00:45:03,916: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-07 00:45:03,918: INFO: common: created directory at: artifacts]
[2024-09-07 00:45:03,919: INFO: common: created directory at: artifacts/training]
